
# Putting it all together

We'll put our client objects in one file...

In [1]:
class TutorRouterClient:
    def __init__(self, hostname: str):
        self.hostname = hostname
    
    @staticmethod
    def instantiate(hostname: str) -> "TutorRouterClient":
        """Instantiate the correct subtype of TutorRouterClient.
        
        Makes a call to the `healthcheck` endpoint to determine
        the right subtype, according to the `version` field.
        
        Args:
            hostname: the router host name
        
        Returns:
            an initialized subtype of `TutorRouterClient`.
        
        Raises:
            Exception: if the router cannot be reached.
            Exception: if the healthcheck cannot be performed.
            Exception: if the version cannot be determined.
        """
        
        # Hit the healthcheck endpoint.
        status_code, response = get(hostname, "healthcheck")
        if status_code != 200:
            raise Exception(
                f"Error whilst performing health check of {hostname}"
            )
        health = json.loads(response)
        
        # Extract the version
        try:
            version = health['version']
        except KeyError:
            raise Exception(
                f"Could not determine version of {hostname}"
            )
        
        # Return an instantiated object of the correct type.
        if version == 1:
            return TutorRouterV1Client(hostname)
        elif version == 2:
            return TutorRouterV2Client(hostname)
        else:
            raise Exception(
                f"Version {version} of {hostname} not known"
            )


class TutorRouterV1Client(TutorRouterClient):
    """A client for a v1 TutorRouter."""
    
    def __init__(self, hostname: str):
        """Initialize a TutorRouterV1Client.
        
        Args:
            hostname: the router's hostname.
        """
        super().__init__(hostname)
        self._auth_token = None

    def login(self, username: str, password: str):
        """Log into the router.
        
        Args:
            username: the user we're logging in.
            password: the user's password.
            
        Raises:
            Exception: if there is an error whilst quthenticating.
        """
        status_code, response = post(
            self.hostname,
            "login",
            headers={"username": username, "password": password}
        )
        if status_code != 200:
            raise Exception(
                f"Error whilst authenticating {username} on {self.hostname}"
            )
        self._auth_token = response
            
    def _get_interface_stats(self) -> dict:
        """Return the interface stats reponse from the router.
        
        Raises:
            Exception: if we get a non-200 response.
            
        Returns:
            See the router API docs.
        """
        status_code, response = get(
            self.hostname,
            "interfaces",
            headers={"Token": self._auth_token}
        )
        if status_code != 200:
            raise Exception(
                "Failed to get interface stats"
            )
        return json.loads(response)
    
    def get_throughput(self) -> int:
        """Return the total throughput (up and down) to the router.
        
        Returns:
            the number of bits per second.
        """
        iface_stats = self._get_interface_stats()
        total_throughput = (
            iface_stats["eth0"]["up"] +
            iface_stats["eth0"]["down"]
        )
        return total_throughput

        
class TutorRouterV2Client(TutorRouterClient):
    """A client for a v2 TutorRouter."""
    
    def __init__(self, hostname: str):
        """Initialize a TutorRouterV2Client.
        
        Args:
            hostname: the router's hostname.
        """
        super().__init__(hostname)
        self._token = None

        
    def login(self, username: str, password: str):
        """Log into the router.
        
        Args:
            username: the user we're logging in.
            password: the user's password.
            
        Raises:
            Exception: if there is an error whilst authenticating.
        """
        status_code, response = post(
            self.hostname,
            f"authenticate/{username}",
            headers={"token": password}
        )
        if status_code != 200:
            raise Exception(
                f"Error whilst authenticating {username} on {self.hostname}"
            )
        self._auth_token = response
    
    def _get_interface_stats(self) -> dict:
        """Return the interface stats reponse from the router.
        
        Raises:
            Exception: if we get a non-200 response.
            
        Returns:
            See the router API docs.
        """
        status_code, response = get(
            self.hostname,
            "interfaces",
            headers={"Authentication": f"TOKEN {self._auth_token}"}
        )
        if status_code != 200:
            raise Exception(
                f"Failed to get interface stats: {response}"
            )
        return json.loads(response)
    
    def get_throughput(self) -> int:
        """Return the total throughput (up and down) to the router.
        
        Returns:
            the number of bits per second as an integer.
        """
        iface_stats = self._get_interface_stats()
        total_throughput = (
            int(iface_stats["eth0"]["up"].split()[0]) * 1000 +
            int(iface_stats["eth0"]["down"].split()[0]) * 1000
        )
        return total_throughput

> There are a few places where repetition could be reduced, especially as more methods are added. Beware, however, of overdoing DRY (Don't Repeat Yourself). Over-DRYness can make code difficult to understand and modify. If you're reaching for reflection/introspection, you've probably gone too far.

### Load the simulation

In [2]:
# Load the libraries
import os, sys, json
sys.path.insert(0, "../src")
from tutor_router.scenarios import Scenario, NotFoundError

# Set up the scenario
scenario = Scenario.scenario_2()

# Make these look a bit like `requests` calls.
request = scenario.request
get = scenario.get
post = scenario.post

# Get a list of host names, for convenience.
hosts = tuple(scenario.hosts())

In [3]:
hosts

('TutorRouter-1',
 'TutorRouter-2',
 'TutorRouter-3',
 'TutorRouter-4',
 'TutorRouter-5',
 'TutorRouter-6',
 'TutorRouter-7',
 'TutorRouter-8',
 'TutorRouter-9',
 'TutorRouter-10',
 'TutorRouter-11',
 'TutorRouter-12',
 'TutorRouter-13',
 'TutorRouter-14',
 'TutorRouter-15',
 'TutorRouter-16',
 'TutorRouter-17',
 'TutorRouter-18',
 'TutorRouter-19',
 'TutorRouter-20')

Now we can use our new object model to handle most of the logic.

Note how there are no checks for firmware version. Each instantiated object knows how to perform its own calls.

In [4]:
# Iterate over all routers.
for hostname in hosts:
    
    # Find out what kind of router it is.
    try:
        router = TutorRouterClient.instantiate(hostname)
    except Exception as exc:
        print(f"Host {hostname} had an error: {exc}")
        continue
    print(f"Host {hostname} is up")

    # Log in
    try:
        router.login("admin", "Password123")
    except Exception as exc:
        print(f"Router {hostname} had a login problem: {exc}")
        continue
    
    # Print the throughput
    print(f"  Throughput: {router.get_throughput()} bits per second")

Host TutorRouter-1 had an error: Connection error
Host TutorRouter-2 is up
  Throughput: 626406 bits per second
Host TutorRouter-3 is up
  Throughput: 1643397 bits per second
Host TutorRouter-4 had an error: Connection error
Host TutorRouter-5 is up
  Throughput: 1225525 bits per second
Host TutorRouter-6 had an error: Connection error
Host TutorRouter-7 is up
  Throughput: 1285145 bits per second
Host TutorRouter-8 had an error: Connection error
Host TutorRouter-9 is up
  Throughput: 895457 bits per second
Host TutorRouter-10 is up
  Throughput: 1004000 bits per second
Host TutorRouter-11 is up
  Throughput: 707000 bits per second
Host TutorRouter-12 is up
  Throughput: 1150618 bits per second
Host TutorRouter-13 is up
  Throughput: 1590000 bits per second
Host TutorRouter-14 is up
  Throughput: 673000 bits per second
Host TutorRouter-15 is up
  Throughput: 875947 bits per second
Host TutorRouter-16 is up
  Throughput: 1192779 bits per second
Host TutorRouter-17 is up
  Throughput: 69

Although the number of lines of code has increased from the class and method declaration overhead, from some repetition here and there, and a lot by adding commentary, the code of the _main loop_ with the _business logic_ is much cleaner and shows none of the underlying logic that we don't care too much about:

* We don't need to pass hostnames around.
* We don't need to pass tokens around.
* We don't need to parse the results of calls.

Especially as our objects gain more capabilities, the advantages of using OOP becomes increasingly evident.

This makes it easier to change the code around. For example, we may want to only run the operations on routers we already know we can get to...

In [5]:
working_routers = []

# Iterate over all routers just to
# instantiate the objects and log in.
for hostname in hosts:

    # Find out what kind of router it is.
    try:
        router = TutorRouterClient.instantiate(hostname)
    except Exception as exc:
        print(f"Host {hostname} had a healthcheck error: {exc}")
        continue

    # Log in
    try:
        router.login("admin", "Password123")
    except Exception as exc:
        print(f"Router {hostname} had a login problem: {exc}")
        continue

    # Only record the ones that worked.
    working_routers.append(router)

    
print("\n\n... some time later...\n\n")


# Only try to gather the throughput
# from routers that worked.
for router in working_routers:
    # Print the throughput
    throughput = router.get_throughput()
    print(f"{router.hostname} throughput: {throughput} bps")
    
    # ... do a load more operations ...

Host TutorRouter-1 had a healthcheck error: Connection error
Host TutorRouter-4 had a healthcheck error: Connection error
Host TutorRouter-6 had a healthcheck error: Connection error
Host TutorRouter-8 had a healthcheck error: Connection error
Host TutorRouter-18 had a healthcheck error: Connection error
Host TutorRouter-19 had a healthcheck error: Connection error
Host TutorRouter-20 had a healthcheck error: Connection error


... some time later...


TutorRouter-2 throughput: 626406 bps
TutorRouter-3 throughput: 1643397 bps
TutorRouter-5 throughput: 1225525 bps
TutorRouter-7 throughput: 1285145 bps
TutorRouter-9 throughput: 895457 bps
TutorRouter-10 throughput: 1004000 bps
TutorRouter-11 throughput: 707000 bps
TutorRouter-12 throughput: 1150618 bps
TutorRouter-13 throughput: 1590000 bps
TutorRouter-14 throughput: 673000 bps
TutorRouter-15 throughput: 875947 bps
TutorRouter-16 throughput: 1192779 bps
TutorRouter-17 throughput: 698525 bps


We can inspect the type of each object like so:

In [6]:
for router in working_routers:
    print(f"{router.hostname}: {type(router)}")

TutorRouter-2: <class '__main__.TutorRouterV1Client'>
TutorRouter-3: <class '__main__.TutorRouterV1Client'>
TutorRouter-5: <class '__main__.TutorRouterV1Client'>
TutorRouter-7: <class '__main__.TutorRouterV1Client'>
TutorRouter-9: <class '__main__.TutorRouterV1Client'>
TutorRouter-10: <class '__main__.TutorRouterV2Client'>
TutorRouter-11: <class '__main__.TutorRouterV2Client'>
TutorRouter-12: <class '__main__.TutorRouterV1Client'>
TutorRouter-13: <class '__main__.TutorRouterV2Client'>
TutorRouter-14: <class '__main__.TutorRouterV2Client'>
TutorRouter-15: <class '__main__.TutorRouterV1Client'>
TutorRouter-16: <class '__main__.TutorRouterV1Client'>
TutorRouter-17: <class '__main__.TutorRouterV1Client'>
